In [2]:
import sys
import pandas as pd
sys.path.append('..')
from khan_project_data import *

pd.set_option('display.max_rows',None)

file = "/Users/chuaixin/Desktop/pypro/static/git_match.xlsx"
df = pd.read_excel(file,sheet_name='流水线明细')

In [3]:
def self_git(ProjectID):
    """
    Purpose:获取项目内代码库 
    """
    conn = pymysql.connect(**conn_khan)
    cursor = conn.cursor()
    project_git_list = '''  
        SELECT
            PV.project_id '项目ID',
            PV.id '版本ID',
            PR.repository_id '代码库ID',
            lower(GITR.`name`) '代码库名称'
	
        FROM
            kh_project_version PV 
            LEFT JOIN kh_project__repository PR ON PV.project_id = PR.project_id
            LEFT JOIN kh_gitlab_repository GITR ON GITR.id = PR.repository_id

        WHERE
	        pv.project_id = {0} AND GITR.`name` is NOT NULL'''.format(ProjectID)
    
    cursor.execute(project_git_list)
    result = cursor.fetchall()
    # 关闭光标对象
    cursor.close()
    conn.close()
    return result
    
# end def

In [10]:
def depend_git(ProjectID):
    """
    Purpose:获取项目内代码库 
    """
    conn = pymysql.connect(**conn_khan)
    cursor = conn.cursor()
    project_git_list = '''  
        SELECT
            PV.project_id '项目ID',
            PV.id '版本ID',
            PVD.depend_project_id '依赖产品ID',
            P.name '依赖产品名称',
            PR.repository_id '代码库ID',
            lower(GITR.`name`) '代码库名称'      
        FROM
            kh_project_version PV 
            LEFT JOIN kh_project_version_dependency PVD ON PVD.project_version_id = PV.id
            LEFT JOIN kh_project__repository PR ON PVD.depend_project_id = PR.project_id
            LEFT JOIN kh_gitlab_repository GITR ON GITR.id = PR.repository_id
            LEFT JOIN kh_project p ON PVD.depend_project_id = p.id 
        WHERE
            pv.project_id = {0} AND p.status !=0 AND GITR.`name` is NOT NULL
        order by PVD.depend_project_id asc; 
            '''.format(ProjectID)
    
    cursor.execute(project_git_list)
    result = cursor.fetchall()
    # 关闭光标对象
    cursor.close()
    conn.close()
    return result
    
# end def

In [5]:
grouped = df.groupby('项目ID')  
  
# 遍历每个分组并打印出来
for pid, group in grouped:
    # print(f"Value in column1: {pid}")
    print(group)
    # print("\n---\n")  # 分隔不同的分组
    show_depend = depend_git(pid)
    print("依赖产品代码库",show_depend)  
    if show_depend:
        df_show_depend = pd.DataFrame(show_depend)  
        # 使用isin()函数检查df_show_depend表的'5'列中的值是否在A表的'代码库名称'列中存在
        # df.loc[group.index, '代码库对应产品'] = df.loc[group.index, '代码库名称'].str.lower().isin(df_show_depend[5]).map({True: '匹配', False: '未匹配'})
        df_depend_match = df_show_depend.loc[df_show_depend[5].isin(df.loc[group.index, '代码库名称'].str.lower()),[5,3]]
        # print("匹配的产品依赖\n",*df_depend_match,sep='\n')
        if df_depend_match.empty:
            df.loc[group.index, '代码库对应产品'] = '有产品关联但无匹配代码库'    
            continue    
        value_mapping = df_depend_match.drop_duplicates(5).set_index(5)  
        value_to_fill = '未匹配' 
        #对df_show的第三列与df_depend_match的索引进行比较，找到匹配的值，并填充到df_show的第四列
        df.loc[group.index, '代码库对应产品']  = df.loc[group.index, '代码库名称'].str.lower().map(value_mapping[3]).fillna(value_to_fill) 
    else:
        df.loc[group.index, '代码库对应产品'] = '无产品关联'   

    print("最终输出：",df.loc[group.index, ['项目ID','代码库名称','代码库对应产品']])   # 添加了这一行，以便查看结果
     

print(df[['项目ID','代码库名称','代码库对应产品']])   # 添加了这一行，以便查看结果    

df.to_excel('/Users/chuaixin/Desktop/pypro/static/git_match_result.xlsx')
    


        一级部门  二级部门  项目ID      项目名称 项目负责人   代码库ID                代码库名称  \
592  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1082.0  cqmams_report_files   
593  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1274.0          UltraBPP-JK   
594  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1275.0         UltraBPP-FJK   
595  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1278.0     MobileServer_new   
596  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1279.0      common_datasync   
597  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1280.0          data_export   
598  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1282.0          dwInterface   
599  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1283.0    zhongbao_intermid   
600  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1284.0      cqmams_intermid   
601  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1296.0          DWMobileAPP   
602  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1298.0          mams-cost-2   
603  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1718.0            dwmonitor   
604  智能运营事业部  项目七部   233  重庆移动代维管理  刘嘉nw  1830.0   

In [8]:
pid = 1100
show = self_git(pid)
# print("self_git:",show) 
show_depend = depend_git(pid)

# print(*show,sep='\n') 
print(*show_depend,sep='\n') 

#通过pandas合并show和show_depend，对比最后一列，标识出匹配情况
# 将 show 和 show_depend 转换为 pandas DataFrame

if show:
    df_show = pd.DataFrame(show)
    if show_depend:
        df_show_depend = pd.DataFrame(show_depend)

        df_depend_match = df_show_depend.loc[df_show_depend[5].isin(df_show[3]),[5,3]]
        print("匹配的产品依赖\n",df_depend_match)

        value_mapping = df_depend_match.set_index(5) 
        value_to_fill = 'Not Found' 
        #对df_show的第三列与df_depend_match的索引进行比较，找到匹配的值，并填充到df_show的第四列
        df_show[4] = df_show[3].map(value_mapping[3]).fillna(value_to_fill)

        # df_show['exists'] = df_show.loc[df_show[3].isin(df_show_depend[5]),3]
        # 将True转换为对应的值 
        # df_show['exists'] = df_show['exists'].map({True: df_show_depend[5][df_show_depend[5].isin(df_show[3])]})
        print("本项目代码库匹配\n",df_show)
    else:
        print("无产品依赖配置")     
else:
    print('无代码库')
# end if


OperationalError: (1054, "Unknown column 'PV.status' in 'where clause'")

<!-- show = depend_git(pid)  

print(*show,sep='\n')  -->